In [ ]:
#Jacob Ballou
#Csci 390
#Modular inflection program
#import
import nltk
import re
import pandas as pd
from nltk import word_tokenize
#rule applier
def applyRule(word,rule):
    #use keyword-based approach
    #split rule into lines
    #print("splitting rules")
    lines = rule.split("\n")
    #for line in lines, go through and apply changes to word
    for line in lines:
        #check to see what the first part is
        changes = line.split("=")
        #print("Splitting line")
        #possible values for changes[0] are prefix, suffix, replace, and none.
        if (changes[0]=="prefix"):
            #add the letters in changes[1] to the front of the word.
            word = changes[1]+word
        elif (changes[0]=="suffix"):
            #add the letters in changes[1] to the end of the word
            word = word+changes[1]
        elif (changes[0]=="replace"):
            #this is the complicated bit.
            #format for replace is as follows:
            #replace=[character(s)],[character(s)],[integer or "all"]
            #split changes[1] into 3.
            params = changes[1].split(",")
            #get occurrance count of original substring
            occurrences = word.count(params[0])
            if params[2] == "all":
                word = word.replace(params[0],params[1])
            #if params[2] is an integer
            elif (re.search("^[-/d]/d+$",params[2])):
                occurrencePos = int(params[2])
                if (occurrencePos < 0):
                    #replace it with the positively indexed equivalent
                    occurrencePos = occurrencePos + occurrences
                #replace the occurrencePos instance of params[0] with params[1]. This part is lifted from stackoverflow-
                #I couldn't find a more simple way to do this.
                pos = [m.start() for m in re.finditer(params[0], word)][occurrencePos-1]
                pre = word[:pos]
                post = word[pos:]
                post = post.replace(params[0],params[1], 1)
                word = pre + post
        elif (changes[0]=="none"):
            #do nothing
            word = word
        else:
            #print that the changes format is wrong
            print("Bad rule format in :\n")
            print(line)
    #loop is finished and all changes are applied to the word.
    return word

#InflectNoun([noun],case,gender,number,[ruleset])
#cases, gender, and number can be any string, as long as they're keys in the dictionary.
#ruleset is a 3 dimensional dictionary, with strings on the bottom layer.
def inflectNoun(noun, case, gender, number,ruleset: dict):
    #check to see if usage is correct
    if ((noun != str)or(case != str)or(gender != str)or(number != str)or (ruleset != dict)):
        print("Usage: inflectNoun(noun = str, case = str, gender = str, number = str, ruleset = dict[case][gender][number])\n")
    else:
        #check ruleset to see if case exists in the dictionary
            if (case in ruleset):
                #check to see if number exists
                if (number in ruleset[case]):
                    #check to see if gender exists
                    if (gender in (ruleset[case])[number]):
                        rule = ruleset[case][number][gender]
                        #apply changes to noun
                        return(applyRule(noun,rule))
                    else:
                        #raise error- gender is not specified for ruleset
                        print("Gender is not specified for ruleset. Try using 'neutral'.\n")
                else:
                    #raise error- number is not specified for ruleset
                    print("Number is not specified for ruleset. Try using 'singular'.\n")
            else:
                #raise error- case is not in ruleset
                print("Noun case is not specified for ruleset. Try using 'nominative'.\n")

#main
def main():
    #import data from Plotz Declension
    #If Jupyter Notebook was working, this would import the data I was planning on using to test the inflector.
    control = "PlotzDeclension.xlsx"
    allDeclensions = pd.read_excel(control)
    #I ran into the problem with JupyterNotebook very early on in the coding process, so I was unable to create a ruleset
    #to test my inflector with. The issue affects both the applyRule function and the inflectNoun function.